# Importing libraries

In [1]:
import joblib

import numpy as np # linear algebra
import pandas as pd # data processing/analys
import lightgbm as lgb # gradient boosting framework that uses tree based learning algorithms
import warnings

from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV

warnings.filterwarnings("ignore")

## Using Pandas to load/read train, test and sample_sub set

In [2]:
train = pd.read_csv('DATA/train.csv')
test = pd.read_csv('DATA/test.csv')
sample_submission = pd.read_csv('DATA/sample_submission.csv')

Droping the lable("revenue") before Scaling.

Saving label as y, using numpy log1p on label(logarithmic element-wise) to get better score.

print(np.expm1(y)) is how we transform back again(necessary after predict to get the right score)

In [3]:
X = train.drop(["revenue"], axis=1)
y = np.log1p(train['revenue'])

A very light weight pipeline with Robust Scaler, which takes care of outliers

In [9]:

pipeline = Pipeline([
    ("Rbt_scaler", RobustScaler())
])

prepared = pipeline.fit_transform(X)


In [10]:
joblib.dump(pipeline, "../model/pipeline.joblib", compress = 1)

['../model/pipeline.joblib']

In [ ]:
params = {'num_leaves': 30,
         'min_data_in_leaf': 20,
         'objective': 'regression',
         'max_depth': 5,
         'learning_rate': 0.01,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9,
         "bagging_seed": 11,
         "metric": 'rmse',
         "lambda_l1": 0.2,
         "verbosity": -1}

lgb_model = lgb.LGBMRegressor(**params, n_estimators = 10000, nthread = 4, n_jobs = -1)


Setting parameters for light gradient boosting model regressor (above)
Using random search to optimize(below), note that I removed all but the best params to save time.

In [ ]:
Random_Search_Params ={
    "max_depth": [4],
    "min_data_in_leaf": [15],
    'learning_rate': [0.01],
    'num_leaves': [40],
    'boosting_type' : ['gbdt'],
    'objective' : ['regression'],
    'random_state' : [501], 
    }

n_HP_points_to_test = 50

random_search = RandomizedSearchCV(
    estimator=lgb_model, param_distributions= Random_Search_Params, 
    n_iter=n_HP_points_to_test,
    cv=3,
    refit=True,
    random_state=314,
    verbose=True)

random_search.fit(prepared, y)

# Using parameters already set above, replace in the best from the random search
params['learning_rate'] = random_search.best_params_['learning_rate']
params['max_depth'] = random_search.best_params_['max_depth']
params['num_leaves'] = random_search.best_params_['num_leaves']



random_search.best_estimator_

In [20]:
joblib.dump(random_search.best_estimator_, "../model/model.joblib", compress = 1)


['../model/model.joblib']

**7 & 8 The solution is launching** 

The app is upp and running! We have a db up on Heroku but when we tried to run the prediction we ran out of ram(and we're too cheap to pay). However its runnable and you get a own movie(predicted revenue is low) where we show the running app. 
One problem is that low budget results in very low revenue this i probably due to that the model uses both budget and budget_year_ratio. Another reason is that movies that where missing budget info(=0) often also had very low revenue.

However most of the time we get a really decent prediction(within 300 000 of the actual revenue), considering how easy to use the app is(only  10 fields) and the small size of the train/test set we are happy. 

People loves movies so there could be an intrest for this app even if it probably wont be a comercial success.

**7 & 8 The solution is launching** 

The app is upp and running! We have a db up on Heroku but when we tried to run the prediction we ran out of ram(and we're too cheap to pay). However its runnable and you get a own movie(predicted revenue is low) where we show the running app. 
One problem is that low budget results in very low revenue this i probably due to that the model uses both budget and budget_year_ratio. Another reason is that movies that where missing budget info(=0) often also had very low revenue.

However most of the time we get a really decent prediction(within 300 000 of the actual revenue), considering how easy to use the app is(only  10 fields) and the small size of the train/test set we are happy. 

People loves movies so there could be an intrest for this app even if it probably wont be a comercial success.